In [6]:
%pip install fastparquet

import pandas as pd

You should consider upgrading via the '/home/ramin/Developer/venv/salsa-evaluation/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [7]:
# Parameters
data = "../../salsa/data/single_partition/partition_0/left_index.gzip"
sample = 500
number_of_interactions = 17
output = "../data/users.json"

In [8]:
# Import users from dataset
keep_columns = [0, 1]

df = pd.read_parquet(data, columns=['adjacency_list', 'user_id'])
df.reset_index(level=0, inplace=True)

In [9]:
df["len_adjacency_list"] = df["adjacency_list"].str.len()
active_users = df[df["len_adjacency_list"] > number_of_interactions]["user_id"]

In [10]:
# Sample users
if sample:
    active_users = active_users.sample(n=sample)

In [11]:
# Save results
active_users.to_json(output, orient="table", index=False)
active_users.reset_index(drop=True)

0      1104808931891793920
1       742613688448864256
2      1345404930094374912
3      1063441789590429696
4               2232404250
              ...         
495     740472581581475840
496               28675507
497              909017503
498    1227709406839619584
499              763209294
Name: user_id, Length: 500, dtype: int64